In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic_settings.sources.providers.yaml import yaml

load_dotenv('my.env')
MODEL = os.getenv("MODEL")
API_KEY = os.getenv("API_KEY")
API_BASE = os.getenv("API_BASE")

llm = ChatOpenAI(
    model=MODEL,
    openai_api_key=API_KEY,
    openai_api_base=API_BASE,
    temperature=0.2,
)


In [21]:
# Пример работы бота, когда few shot не используется
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = PromptTemplate(
    template=(
        "Ты опытный программист на python. Твоя задача дать краткую справку, как реализовать функцию на python"
        "Формат ответа:  описание, как решить проблему. "
        "Вопрос:{user_question}. Ответ:"
    ),
    input_variables=["user_question"]
)

chain = prompt | llm | StrOutputParser()

In [22]:
result = chain.invoke({"user_question": "Как отсортировать список объектов по полю в Python?"})
print(result)

Чтобы отсортировать список объектов по определенному полю в Python, можно использовать функцию `sorted()` с параметром `key`. Вот пример:

```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

people = [Person("Alice", 30), Person("Bob", 25), Person("Charlie", 35)]

# Сортировка по возрасту (возрастание)
sorted_people = sorted(people, key=lambda x: x.age)

# Сортировка по имени (убывание)
sorted_people_by_name = sorted(people, key=lambda x: x.name, reverse=True)
```

**Ключевые моменты:**
1. Используйте `key=lambda x: x.поле` для указания поля сортировки.
2. Параметр `reverse=True` позволяет сортировать в обратном порядке.
3. Для сортировки на месте используйте метод `.sort()` вместо `sorted()`.

Если объекты имеют атрибуты с динамическими именами, можно использовать `operator.attrgetter`:
```python
from operator import attrgetter

sorted_people = sorted(people, key=attrgetter('age'))
```

Для сложных случаев (например, сортировка п

In [23]:
# Пример работы бота, когда few shot используется
from langchain_core.prompts import FewShotPromptTemplate
import yaml

def load_prompts(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

data = load_prompts('home_work_2.5_prompts.yaml')

examples = data['prompts']

examples_prompt = PromptTemplate(
    template='Вопрос: {{question}}\nОтвет: {{answer}}',
    input_variables=['question', 'answer'],
    template_format='jinja2'
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=examples_prompt,
    prefix="Ты опытный программист на python. Твоя задача дать краткую справку, как реализовать функцию на python. "
        "Формат ответа: описание, как решить проблему.",
    suffix="Вопрос: {{user_question}}\nОтвет:",
    input_variables=["user_question"],
    template_format='jinja2'
)

In [24]:
chain = prompt | llm | StrOutputParser()
result = chain.invoke({"user_question": "Как отсортировать список объектов по полю в Python?"})
print(result)

Чтобы отсортировать список объектов по определенному полю, можно использовать функцию `sorted()` с параметром `key`. Вот пример:

```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

people = [Person("Alice", 30), Person("Bob", 25), Person("Charlie", 35)]

# Сортировка по возрасту (возрастание)
sorted_people = sorted(people, key=lambda x: x.age)

for person in sorted_people:
    print(person.name, person.age)
```

Результат:
```
Bob 25
Alice 30
Charlie 35
```

Если нужно больше примеров или сортировка по убыванию, сообщите.
